# LASSO regression of FI vs. GO terms TF-IDF

## Preparation

In [3]:
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn import linear_model
import matplotlib.pyplot as plt

### Import GO data

In [4]:
with open('../processed_data/geneOntologyDataset.pkl', 'rb') as inp:
    [pivoted, tfidfPivoted] = pkl.load(inp)

In [5]:
GOdata = pd.DataFrame(tfidfPivoted['P'].todense()) #.join(pd.DataFrame(tfidfPivoted['P'].todense()), lsuffix='_left', rsuffix='_right').join(pd.DataFrame(tfidfPivoted['C'].todense()), lsuffix='_left2', rsuffix='_right2') # F / P / C
GOdata["Standard Name"] = pivoted['P'].index

### Import FI data & merge

In [6]:
FIdata = pd.read_excel('../raw_data/41592_2018_45_MOESM4_ESM.xlsx', skiprows = 6, engine = 'openpyxl')
full_data = pd.merge(GOdata, FIdata, on = 'Standard Name', how = 'left')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
full_data.index = full_data["Systematic Name"]
full_data
x = full_data.iloc[:, 0:full_data.columns.get_loc('Standard Name')]
x.columns = pivoted.get("P").columns
y = full_data["intensity"]

In [9]:
y = y[y.notna()]

In [10]:
xy = pd.merge(x, y, how = "right", left_index = True, right_index = True)

## LASSO

In [11]:
clf = linear_model.LassoLarsIC(criterion = 'bic', normalize = False, eps = 1e-7)

In [12]:
clf.fit(xy.iloc[:,0:-1], xy.iloc[:,-1])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:681: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 173 iterations, alpha=3.831e-04, previous alpha=3.831e-04, with an active set of 162 regressors.
  warnings.warn(


LassoLarsIC(criterion='bic', eps=1e-07, normalize=False)

In [20]:
# number of GO terms with nonzero coefficient
np.sum([clf.coef_ != 0])

59

In [21]:
res = pd.DataFrame(clf.coef_)
res.index = x.columns

In [26]:
selection = res[res[0] != 0]
selection.sort_values(by = 0)

,0
goId,
GO:0071472,-1.082521
GO:0006468,-0.401983
GO:0055085,-0.364835
GO:0006355,-0.189929
GO:0016567,-0.168873
GO:0007049,-0.159631
GO:0000122,-0.154924
GO:0006397,-0.133933
GO:0051321,-0.127285


In [25]:
# R^2 of the model
clf.score(xy.iloc[:,0:-1], xy.iloc[:,-1])

0.511620664265426